In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium

In [2]:
spark = SparkSession.builder.master('local[*]') \
    .appName('Métricas') \
    .getOrCreate()

23/09/30 14:56:56 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/30 14:56:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/30 14:56:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
spark.stop()

NameError: name 'spark' is not defined

<p>1 – Quantidade de ônibus em operação</p>

In [4]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_agrupada/"
file_list = os.listdir(directory_path)
dataframes_agrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)
        df = spark.read.parquet(caminho_completo)
        dataframes_agrupados.append(df)

In [5]:
def union_all(dfs):
    return reduce(DataFrame.unionAll, dfs)

In [6]:
dataframes_agrupados_completo = union_all(dataframes_agrupados)
dataframes_agrupados_completo.show(truncate=False)

+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|LINHA  |CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM  |LETREIRO_DESTINO     |QTDE_VEICULOS_OPERACAO|DATA_EXTRACAO_API|
+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|3054-10|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |08:50   |2023-09-15   |3054-10          |33743               |HOSP. SAPOPEMBA  |JD. PALANQUE         |6                     |2023-09

In [40]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)
 |-- TURNO: string (nullable = true)



In [7]:
dataframes_agrupados_completo.count()

1083996

In [8]:
lista_tempo = dataframes_agrupados_completo.select('DATA_EXTRACAO')\
        .distinct().rdd.flatMap(lambda linha: [linha.DATA_EXTRACAO]).collect()

In [9]:
lista_tempo

[datetime.date(2023, 9, 15),
 datetime.date(2023, 9, 16),
 datetime.date(2023, 9, 17),
 datetime.date(2023, 9, 18)]

In [10]:
max(lista_tempo)

datetime.date(2023, 9, 18)

In [11]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)
 |-- DATA_EXTRACAO_API: date (nullable = true)



In [12]:
dataframes_agrupados_completo = dataframes_agrupados_completo.drop('DATA_EXTRACAO_API')
dataframes_agrupados_completo.show()

+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|  LETREIRO_ORIGEM|    LETREIRO_DESTINO|QTDE_VEICULOS_OPERACAO|
+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|3054-10|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   08:50|   2023-09-15|          3054-10|               33743|  HOSP. SAPOPEMBA|        JD. PALANQUE|                     6|
|6026-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   08:50|   2023-09-15|          6026-10|                  87| TERM. STO. AMARO|          JD. ICARAÍ|                     5|
|857A-10|          7|      CONSÓRCIO KBPX|KBPX ADMINISTRAÇÃ.

In [13]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)



In [14]:
dataframes_agrupados_completo.columns

['LINHA',
 'CODIGO_AREA',
 'CONSOCIO',
 'EMPRESA',
 'HORA_API',
 'DATA_EXTRACAO',
 'LETREIRO_COMPLETO',
 'CODIGO_IDENTIFICADOR',
 'LETREIRO_ORIGEM',
 'LETREIRO_DESTINO',
 'QTDE_VEICULOS_OPERACAO']

In [15]:
@f.udf(returnType=t.StringType())
def turno(hora: str):
    hora_formatada = datetime.strptime(hora, '%H:%M').time()
    if 0 <= hora_formatada.hour < 6:
        return 'Madrugada'
    elif 6 <= hora_formatada.hour < 12:
        return 'Manhã'
    elif 12 <= hora_formatada.hour < 18:
        return 'Tarde'
    else:
        return 'Noite'
        

In [16]:
turno('23:55')

Column<'turno(23:55)'>

In [17]:
dataframes_agrupados_completo = dataframes_agrupados_completo.withColumn('TURNO', turno(f.col('HORA_API'))) 

df_filter = dataframes_agrupados_completo.select(
    dataframes_agrupados_completo.CODIGO_AREA,
    dataframes_agrupados_completo.CONSOCIO,
    dataframes_agrupados_completo.EMPRESA,
    dataframes_agrupados_completo.TURNO,
    dataframes_agrupados_completo.DATA_EXTRACAO ,
    dataframes_agrupados_completo.LETREIRO_ORIGEM,
    dataframes_agrupados_completo.LETREIRO_DESTINO,
    dataframes_agrupados_completo.HORA_API,
    dataframes_agrupados_completo.LETREIRO_COMPLETO,
    dataframes_agrupados_completo.QTDE_VEICULOS_OPERACAO,
    dataframes_agrupados_completo.CODIGO_IDENTIFICADOR,
    ) \
    .filter(
        (dataframes_agrupados_completo.DATA_EXTRACAO == '2023-09-16' )  &
        
        (dataframes_agrupados_completo.TURNO == 'Noite') &
        (dataframes_agrupados_completo.LETREIRO_COMPLETO == '9047-10')
        ) \
        
df_filter.show(truncate=False)

+-----------+-----------------------+---------------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|CODIGO_AREA|CONSOCIO               |EMPRESA                    |TURNO|DATA_EXTRACAO|LETREIRO_ORIGEM|LETREIRO_DESTINO|HORA_API|LETREIRO_COMPLETO|QTDE_VEICULOS_OPERACAO|CODIGO_IDENTIFICADOR|
+-----------+-----------------------+---------------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|1          |CONSÓRCIO TRANSNOROESTE|NORTE BUSS TRANSPORTES S.A.|Noite|2023-09-16   |LAPA           |JD. PAULISTANO  |18:05   |9047-10          |4                     |33651               |
|1          |CONSÓRCIO TRANSNOROESTE|NORTE BUSS TRANSPORTES S.A.|Noite|2023-09-16   |LAPA           |JD. PAULISTANO  |18:05   |9047-10          |3                     |883                 |
|1          |CONSÓRCIO TRANSNOROESTE|NORTE BUSS TR

In [18]:
df_filter.groupBy(['CODIGO_AREA']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULOS_OPERACAO')
    ) \
    .orderBy('CODIGO_AREA') \
    .show(truncate=False)
df_filter.toPandas()

+-----------+----------------------------+
|CODIGO_AREA|QUANTIDADE_VEICULOS_OPERACAO|
+-----------+----------------------------+
|1          |357                         |
+-----------+----------------------------+



,CODIGO_AREA,CONSOCIO,EMPRESA,TURNO,DATA_EXTRACAO,LETREIRO_ORIGEM,LETREIRO_DESTINO,HORA_API,LETREIRO_COMPLETO,QTDE_VEICULOS_OPERACAO,CODIGO_IDENTIFICADOR
0,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:05,9047-10,4,33651
1,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:05,9047-10,3,883
2,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:10,9047-10,5,33651
3,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:10,9047-10,3,883
4,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:15,9047-10,3,33651
...,...,...,...,...,...,...,...,...,...,...,...
132,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:45,9047-10,2,33651
133,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:45,9047-10,1,883
134,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:50,9047-10,2,33651
135,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:55,9047-10,1,33651


In [19]:
df_filter.groupBy(['EMPRESA']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULO_OPERACAO')
    ) \
    .show(truncate=False)

+---------------------------+---------------------------+
|EMPRESA                    |QUANTIDADE_VEICULO_OPERACAO|
+---------------------------+---------------------------+
|NORTE BUSS TRANSPORTES S.A.|357                        |
+---------------------------+---------------------------+



In [20]:
df_filter.groupBy(['DATA_EXTRACAO', 'EMPRESA']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULO_OPERACAO') 
    ) \
    .show(truncate=False)

+-------------+---------------------------+---------------------------+
|DATA_EXTRACAO|EMPRESA                    |QUANTIDADE_VEICULO_OPERACAO|
+-------------+---------------------------+---------------------------+
|2023-09-16   |NORTE BUSS TRANSPORTES S.A.|357                        |
+-------------+---------------------------+---------------------------+



In [21]:
df_filter.groupBy(['DATA_EXTRACAO']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULO_OPERACAO') 
    ) \
    .show(truncate=False)

+-------------+---------------------------+
|DATA_EXTRACAO|QUANTIDADE_VEICULO_OPERACAO|
+-------------+---------------------------+
|2023-09-16   |357                        |
+-------------+---------------------------+



In [22]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_desagrupada"
file_list = os.listdir(directory_path)
dataframes_desagrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)

        df = spark.read.parquet(caminho_completo)
        dataframes_desagrupados.append(df)



In [62]:
dataframes_desagrupados_completo = union_all(dataframes_desagrupados)
dataframes_desagrupados_completo = dataframes_desagrupados_completo.drop('LINHA')
dataframes_desagrupados_completo = dataframes_desagrupados_completo.withColumn('TURNO', turno(f.col('HORA_API')))
dataframes_desagrupados_completo.show(truncate=False)

+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |DATA_EXTRACAO_API|TURNO|
+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |2023-09-15 

<p>Seleção por Id ônibus</p>

In [70]:
df_prefixo_onibus = dataframes_desagrupados_completo.select(
        dataframes_desagrupados_completo.PREFIXO_ONIBUS,
    ).distinct().rdd.flatMap(lambda linha: [linha.PREFIXO_ONIBUS]).collect()
df_prefixo_onibus

[66580,
 15173,
 73951,
 78598,
 26486,
 61679,
 36102,
 31944,
 66070,
 11567,
 48280,
 22129,
 31703,
 15057,
 26543,
 77605,
 36103,
 45298,
 55474,
 66114,
 63212,
 31156,
 64442,
 71655,
 32098,
 68463,
 56242,
 52743,
 72935,
 52001,
 10422,
 61749,
 71530,
 48899,
 51418,
 21965,
 63676,
 52910,
 51428,
 64317,
 31762,
 11745,
 51056,
 72485,
 15194,
 45166,
 72926,
 45410,
 25207,
 16530,
 55426,
 45726,
 71429,
 16597,
 85760,
 52051,
 71285,
 71292,
 47745,
 71386,
 81572,
 36077,
 47050,
 71549,
 16673,
 81443,
 48963,
 52904,
 76085,
 77441,
 71548,
 45390,
 26140,
 85641,
 15221,
 68166,
 68047,
 63181,
 63448,
 86153,
 66911,
 73916,
 71255,
 51105,
 68752,
 76050,
 11771,
 47741,
 48203,
 63678,
 45724,
 63586,
 81038,
 55329,
 55093,
 77434,
 11860,
 45558,
 68314,
 51112,
 72928,
 31159,
 32204,
 47683,
 15133,
 11988,
 31111,
 47636,
 63109,
 47229,
 78203,
 52745,
 22808,
 31126,
 68608,
 45134,
 31102,
 80978,
 31832,
 36577,
 45473,
 16499,
 22707,
 45580,
 45474,


In [173]:
dataframe_prefixo_onibus = dataframes_desagrupados_completo \
    .select(
        dataframes_desagrupados_completo.HORA_API,
        dataframes_desagrupados_completo.PREFIXO_ONIBUS,
        dataframes_desagrupados_completo.CODIGO_IDENTIFICADOR_LINHA,
        dataframes_desagrupados_completo.LETREIRO_COMPLETO,
        dataframes_desagrupados_completo.LATITUDE,
        dataframes_desagrupados_completo.LONGITUDE,
    ) \
    .filter(
        (dataframes_desagrupados_completo.PREFIXO_ONIBUS == '68712') &
        (dataframes_desagrupados_completo.TURNO == 'Noite') &
        (dataframes_desagrupados_completo.DATA_EXTRACAO == '2023-09-18')
    )



dataframe_prefixo_onibus.show(33)

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         68712|                     33040|          546T-10|        -23.6541035|-46.710938999999996|
|   18:05|         68712|                     33040|          546T-10|       -23.65393525|-46.707456500000006|
|   18:10|         68712|                     33040|          546T-10|        -23.6543535|      -46.701634625|
|   18:15|         68712|                     33040|          546T-10|       -23.65908975|        -46.7012395|
|   18:20|         68712|                     33040|          546T-10|        -23.6613215|         -46.693996|
|   18:25|         68712|                     33040|          546T-10|         -23.670195|         -46.688707|
|

<p>Obtendo o vinculo de ônibus e a linha</p>

In [189]:
prefixo_onibus = dataframe_prefixo_onibus.select(
    dataframe_prefixo_onibus.LETREIRO_COMPLETO
) \
.distinct().collect()[0][0]
prefixo_onibus

'546T-10'

<p>Obter dado Linha</p>

In [175]:
df_selecao_rota_linha = pd.read_csv('../data/datalake/bronze/arquivos_gtfs/routes.txt', ) 
df_selecao_rota_linha.head()

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,1012-10,1,1012-10,Term. Jd. Britania - Jd. Monte Belo,3,509E2F,FFFFFF
1,1012-21,1,1012-21,Term. Jd. Britânia - Jd. Rosinha,3,509E2F,FFFFFF
2,1015-10,1,1015-10,Term. Jd. Britânia - Chác. Maria Trindade,3,509E2F,FFFFFF
3,1016-10,1,1016-10,Cem. Do Horto - Shop. Center Norte,3,002F6C,FFFFFF
4,1017-10,1,1017-10,Perus - Conexão Vl. Iório,3,509E2F,FFFFFF


In [141]:
df_selecao_rota_linha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322 entries, 0 to 1321
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   route_id          1322 non-null   object
 1   agency_id         1322 non-null   int64 
 2   route_short_name  1322 non-null   object
 3   route_long_name   1322 non-null   object
 4   route_type        1322 non-null   int64 
 5   route_color       1322 non-null   object
 6   route_text_color  1313 non-null   object
dtypes: int64(2), object(5)
memory usage: 72.4+ KB


In [176]:
df_selecao_rota_linha['route_id'] = df_selecao_rota_linha['route_id'].astype('string')
df_selecao_rota_linha['route_short_name'] = df_selecao_rota_linha['route_short_name'].astype('string')
df_selecao_rota_linha['route_long_name'] = df_selecao_rota_linha['route_long_name'].astype('string')
df_selecao_rota_linha['route_color'] = df_selecao_rota_linha['route_color'].astype('string')
df_selecao_rota_linha['route_text_color'] = df_selecao_rota_linha['route_text_color'].astype('string')

In [143]:
df_selecao_rota_linha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322 entries, 0 to 1321
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   route_id          1322 non-null   string
 1   agency_id         1322 non-null   int64 
 2   route_short_name  1322 non-null   string
 3   route_long_name   1322 non-null   string
 4   route_type        1322 non-null   int64 
 5   route_color       1322 non-null   string
 6   route_text_color  1313 non-null   string
dtypes: int64(2), string(5)
memory usage: 72.4 KB


In [144]:
prefixo_onibus

'6053-10'

In [177]:

df_selecao_rota_linha = df_selecao_rota_linha.query(f'route_id == "{prefixo_onibus}"')
lista_onibus = df_selecao_rota_linha[['route_id', 'route_long_name', 'route_color']].values.tolist()
lista_onibus[0]


['546T-10', 'Vl. Guacuri - Term. Sto. Amaro', '0082BA']

In [160]:
*outos, outros2 = lista_onibus[0]
outros2, outos

('0082BA', ['6053-10', 'Jd. Ellus - Term. Grajaú'])

<p>Obter dados distâncias</p>

In [178]:
def haversine(lat1, lon1, lat2, lon2):

    r = 6371.0
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

 
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = r * c

    return distance

@pandas_udf(DoubleType())
def haversine_udf(lat, lon):
    distances = []
    for i in range(len(lat)):
        
        if i == 0:
            distances.append(0.0)  
        else:
            distance = haversine(lat[i], lon[i], lat[i - 1], lon[i - 1])
            distances.append(distance)
    return pd.Series(distances)

df= dataframe_prefixo_onibus.withColumn("DISTANCIA", haversine_udf(dataframe_prefixo_onibus["LATITUDE"], dataframe_prefixo_onibus["LONGITUDE"]))


df.show(33)






+--------+--------------+--------------------------+-----------------+-------------------+-------------------+--------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|           DISTANCIA|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+--------------------+
|   18:00|         68712|                     33040|          546T-10|        -23.6541035|-46.710938999999996|                 0.0|
|   18:05|         68712|                     33040|          546T-10|       -23.65393525|-46.707456500000006|  0.3551956621027376|
|   18:10|         68712|                     33040|          546T-10|        -23.6543535|      -46.701634625|  0.5947951106073452|
|   18:15|         68712|                     33040|          546T-10|       -23.65908975|        -46.7012395|   0.528182349704729|
|   18:20|         68712|                     33040|          546T-10|      

In [179]:
soma_distancias = df.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')

Soma das distâncias: 11.84 km


In [180]:
posicao_onibus = df.rdd.flatMap(
    lambda linha: [[linha.LATITUDE, linha.LONGITUDE]]
).collect()

posicao_onibus

[[-23.6541035, -46.710938999999996],
 [-23.65393525, -46.707456500000006],
 [-23.6543535, -46.701634625],
 [-23.65908975, -46.7012395],
 [-23.6613215, -46.693996],
 [-23.670195, -46.688707],
 [-23.677691000000003, -46.685349],
 [-23.6816385, -46.682483000000005],
 [-23.689941, -46.6788145],
 [-23.695130499999998, -46.655341],
 [-23.695213000000003, -46.6539025],
 [-23.6997765, -46.645726499999995],
 [-23.701917, -46.639164],
 [-23.703032, -46.631026000000006],
 [-23.7031815, -46.625156000000004],
 [-23.704041500000002, -46.6264175],
 [-23.704088499999997, -46.6266995],
 [-23.704049, -46.626543],
 [-23.7036385, -46.627480000000006]]

In [181]:
mapa_linhas = folium.Map(location = posicao_onibus[0], 
                         zoom_start = 12, 
                         control_scale = True)



for posicao in posicao_onibus:
    folium.Marker(
        location=posicao,  
        popup='This is a marker!',  
        icon=folium.Icon(color='blue')  
    ).add_to(mapa_linhas)

    
mapa_linhas.add_child(folium.LatLngPopup())

mapa_linhas

In [183]:
df_selecao_linha = spark.read.options(delimiter=',', header=True, inferSchema='True') \
        .csv('../data/datalake/bronze/arquivos_gtfs/trips.txt', )

df_selecao_linha = df_selecao_linha.select(
    df_selecao_linha.shape_id
).filter(
    df_selecao_linha.route_id == prefixo_onibus
)

shape_id = df_selecao_linha.rdd.flatMap(lambda linha: [linha.shape_id]).collect()
shape_id

[80054, 80055]

In [184]:
df_selecao_linha.show()

+--------+
|shape_id|
+--------+
|   80054|
|   80055|
+--------+



In [185]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

In [186]:
df_trajeto_linhas = spark.read.options(delimiter=',', header=True).csv('../data/datalake/bronze/arquivos_gtfs/shapes.txt', )
df_linha_filter = df_trajeto_linhas.filter(df_trajeto_linhas.shape_id.isin(shape_id) )
df_linha_filter = df_linha_filter.withColumn('shape_pt_lat', col('shape_pt_lat').cast(FloatType()))
df_linha_filter = df_linha_filter.withColumn('shape_pt_lon', col('shape_pt_lon').cast(FloatType()))
posicao = df_linha_filter.rdd.flatMap(lambda linha: [[linha.shape_pt_lat, linha.shape_pt_lon]]).collect()
posicao
# df_linha_filter.show()

[[-23.70367431640625, -46.62740707397461],
 [-23.703601837158203, -46.6275520324707],
 [-23.703487396240234, -46.62775421142578],
 [-23.703420639038086, -46.62786102294922],
 [-23.703350067138672, -46.627952575683594],
 [-23.703208923339844, -46.62811279296875],
 [-23.70313835144043, -46.62814712524414],
 [-23.70307731628418, -46.6281852722168],
 [-23.70297622680664, -46.62822341918945],
 [-23.702844619750977, -46.62828063964844],
 [-23.702735900878906, -46.62838363647461],
 [-23.702913284301758, -46.62864685058594],
 [-23.703115463256836, -46.62895584106445],
 [-23.703189849853516, -46.62913513183594],
 [-23.703142166137695, -46.62924575805664],
 [-23.703092575073242, -46.6294059753418],
 [-23.703079223632812, -46.62953567504883],
 [-23.70307159423828, -46.629730224609375],
 [-23.70306396484375, -46.62995147705078],
 [-23.703014373779297, -46.63053894042969],
 [-23.703012466430664, -46.63084030151367],
 [-23.703018188476562, -46.630916595458984],
 [-23.703046798706055, -46.63113403320

In [187]:
posicao[0]

[-23.70367431640625, -46.62740707397461]

In [188]:
mapa_linhas = folium.Map(location = posicao[0], 
                         zoom_start = 12, 
                         control_scale = True)



for i in range(len(posicao)-1):
    folium.PolyLine(locations = [[posicao[i][0], posicao[i][1]], [posicao[i+1][0], posicao[i+1][1]]],
                   color = 'red').add_to(mapa_linhas)
    
mapa_linhas.add_child(folium.LatLngPopup())

mapa_linhas